In [9]:
import io
import pandas as pd
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.files.file import File

## -- Variables & connections
username = ''
password = ''
url = ''
#funciones
def connect_to_sharepoint(url_file):
   ctx_auth = AuthenticationContext(url_file)
   if ctx_auth.acquire_token_for_user(username, password):
       ctx = ClientContext(url_file, ctx_auth)
       web = ctx.web
       ctx.load(web)
       ctx.execute_query()
   response = File.open_binary(ctx, url_file)
   bytes_file_obj = io.BytesIO()
   bytes_file_obj.write(response.content)
   bytes_file_obj.seek(0)
   return(bytes_file_obj)
#body
df = pd.read_excel(connect_to_sharepoint(url), header=0)

In [ ]:
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.sharepoint.client_context import ClientContext
import os

#Sharepoint Variables
site_url = 'https://company.sharepoint.com/sites/'
username = ''
password = "" 
folder_url_destination = '/sites/'
#Local Variables
local_directory = r'C:\Users\username\folder_name' 
file_name = 'file_name' + '.xlsx'
source_path = os.path.join(local_directory, file_name)

ctx_auth = AuthenticationContext(site_url)
if ctx_auth.acquire_token_for_user(username, password):
    ctx = ClientContext(site_url, ctx_auth)
    folder = ctx.web.get_folder_by_server_relative_url(folder_url_destination)  

    with open(source_path, 'rb') as file_content:
        file_name = source_path.split(f'\\')[-1]
        file_creation_info = {
            'FileLeafRef': file_name,
            'Content': file_content.read()
        }
    #print(f'File {file_name} ')
    # Upload the file
    upload_file = folder.upload_file(file_name, file_creation_info['Content']).execute_query()
    print(f'*** - File {file_name} has been uploaded to {folder_url_destination}')
else:
    print('Authentication failed')